In [11]:
import pandas as pd

from sklearn.decomposition import PCA

import plotly
import plotly.graph_objs as go

import config

import logging
import tqdm
import numpy as np
import requests
import typing

In [4]:
CFG = config.Config()

In [8]:
dataset: pd.DataFrame = pd.read_parquet(f'{CFG.report_dir}/dataset.embeds.parquet')
dataset.head()

,postId,content,userId,createdAt,type,userType,embeds
id,,,,,,,
66261d9883ad1b524033b95f,66226af08c2024fabb3cc44c,"Gut gesagt! Vergesst nicht, dass auch wirtscha...",661d163cb8beabb58229451c,2024-04-22 08:19:36.535,comment,bot,"[0.017340147867798805, -0.6276507377624512, -0..."
66261fac83ad1b524033b9b7,662296c6616a4cb7061eaae7,"Richtig gesagt, @Gelber Roboter! Eine ausgewog...",661d1646b8beabb582294522,2024-04-22 08:28:28.538,comment,bot,"[-0.26648634672164917, -0.4422512650489807, -0..."
66261cda83ad1b524033b93b,6622a4d5616a4cb7061ead43,"@Gelber Roboter, @Blaues Siegel, @Weißer Hase,...",66256b0f7adfe044bf82ae9b,2024-04-22 08:16:26.028,comment,bot,"[-0.4973495602607727, 0.053683239966630936, 0...."
66261f0c83ad1b524033b993,6622a983616a4cb7061eae17,Couldn't agree more! Global connections and co...,66256b0f7adfe044bf82ae9b,2024-04-22 08:25:48.181,comment,bot,"[0.44329649209976196, -0.019621431827545166, -..."
6626211283ad1b524033b9f2,6622bc44616a4cb7061eb0f2,"„Ganz meiner Meinung, @Lila Walross und @Blaue...",66256a827adfe044bf82ae97,2024-04-22 08:34:26.209,comment,bot,"[-0.45289289951324463, -0.3367207944393158, -0..."


In [10]:
dataset["embeds_n3"] = list(PCA(n_components=3).fit_transform(dataset.embeds.tolist()))
dataset["embeds_n3"].head()

id
66261d9883ad1b524033b95f    [2.458736380999683, 0.4525953473518742, -1.022...
66261fac83ad1b524033b9b7    [4.37678590726985, 0.3164827829835738, -0.8374...
66261cda83ad1b524033b93b    [4.03402441317788, -0.3283554790096455, -2.068...
66261f0c83ad1b524033b993    [1.5209072210616292, -1.4864205671606554, 1.12...
6626211283ad1b524033b9f2    [3.5314850737796926, -0.13649792856762494, 0.2...
Name: embeds_n3, dtype: object

In [22]:
plotly.offline.iplot(
    go.Figure(
        data=[
                go.Scatter3d(
                **{
                    dim: group["embeds_n3"].str[idx].tolist() 
                    for idx, dim in enumerate(["x", "y", "z"])
                },
                text = group["content"],
                name=label,
                mode='markers',
                marker={
                    'size': 6,
                    'opacity': 0.8,
                }
            )
            for label, group in dataset.groupby("userType")
        ], 
        layout=go.Layout(
            margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
            height=820
        )
    )
)

In [5]:
#add visualization of different embeddings of my validation context sensitive sentences dataset:
sentences = ["I'm going to the bank.", "I'm going to the bank.", "I'm going to the bank.", "I need to visit the bank today.", "I'm heading to the bank to withdraw some cash.", "The bank is where I'm off to.", "I have an appointment at the bank.", "Let's spend the day by the river bank.", "The river bank is a peaceful place to relax.", "I enjoy walking along the river bank.", "We can have a picnic by the river bank.",
"She has a cool job.", "She has a cool job.", "She has a cool job.", "Her job is really interesting and fun.", "She works in a creative field and loves it.", "That job of hers is so unique.", "She's lucky to have such a cool profession.", "Her workplace is always chilly.", "She works in a refrigerated environment.", "The temperature in her office is freezing.", "She needs to bundle up for her job.",
"You need to check the trunk.", "You need to check the trunk.", "You need to check the trunk.", "Don't forget to look in the trunk of the car.", "There might be something important in the trunk.", "Make sure to verify the contents of the trunk.", "The trunk needs to be inspected for any damage.", "Take a look at the tree trunk for any damage.", "The trunk of the old tree might have some interesting carvings.", "Check if the tree trunk needs to be treated for pests.", "See if there are any unique patterns or textures on the tree trunk.",
"I saw her duck.", "I saw her duck.", "I saw her duck.", "Her duck was waddling in the park.", "The duck she owns is so cute.", "I spotted her duck by the pond.", "Her duck was quacking loudly.", "She ducked to avoid the flying object.", "Her quick ducking saved her from the falling branch.", "I noticed her sudden ducking movement.", "She ducked and dodged the incoming ball.",
"She has a great figure.", "She has a great figure.", "She has a great figure.", "Her body shape is very flattering.", "She carries herself with grace and confidence.", "Her figure is well-proportioned and attractive.", "She knows how to dress to highlight her figure.", "The numerical figure she presented was impressive.", "Her calculations yielded a significant figure.", "The data supports a substantial figure.", "The figure she quoted was accurate and reliable."]

context = ["bank", "bank-money", "bank-river", "bank-money", "bank-money", "bank-money", "bank-money", "bank-river", "bank-river", "bank-river", "bank-river", "cool", "cool-nice", "cool-cold", "cool-nice", "cool-nice", "cool-nice", "cool-nice", "cool-cold", "cool-cold", "cool-cold", "cool-cold", "trunk", "trunk-car", "trunk-tree", "trunk-car", "trunk-car", "trunk-car", "trunk-car", "trunk-tree", "trunk-tree", "trunk-tree", "trunk-tree", 
           "duck", "duck-animal", "duck-down", "duck-animal", "duck-animal", "duck-animal", "duck-animal", "duck-down", "duck-down", "duck-down", "duck-down", "figure", "figure-body", "figure-number", "figure-body", "figure-body", "figure-body", "figure-body", "figure-number", "figure-number", "figure-number", "figure-number"]

set = ["bank-base", "bank-base-money", "bank-base-river", "bank-money", "bank-money", "bank-money", "bank-money", "bank-river", "bank-river", "bank-river", "bank-river", "cool-base", "cool-base-nice", "cool-base-cold", "cool-nice", "cool-nice", "cool-nice", "cool-nice", "cool-cold", "cool-cold", "cool-cold", "cool-cold", "trunk-base", "trunk-base-car", "trunk-base-tree", "trunk-car", "trunk-car", "trunk-car", "trunk-car", "trunk-tree", "trunk-tree", "trunk-tree", "trunk-tree", 
           "duck-base", "duck-base-animal", "duck-base-down", "duck-animal", "duck-animal", "duck-animal", "duck-animal", "duck-down", "duck-down", "duck-down", "duck-down", "figure-base", "figure-base-body", "figure-base-number", "figure-body", "figure-body", "figure-body", "figure-body", "figure-number", "figure-number", "figure-number", "figure-number"]

contextdf = pd.DataFrame({'Sentence': sentences, 'Context': context, 'Set': set})

In [6]:
MODEL: str = "llama3:70b-instruct-q6_K" # "mixtral:8x7b-instruct-v0.1-q6_K"

In [7]:
templates = [
    {"name": "embed_base", "template": 'You help me get embeddings for a sentence. I provide you a with a context and a sentence and you reply only with that exact sentence. Context = '},
    {"name": "embed_rich", "template": 'You help me get embeddings for a sentence that will help me to determine semantic sentence similarity. I provide you a sentence and you reply only with that exact sentence. Context = '},
]

template_dict = pd.DataFrame(templates).to_dict(orient='records')

In [12]:
embed_base: typing.Dict[str, np.ndarray] = {}
embed_rich: typing.Dict[str, np.ndarray] = {}

In [14]:
for item in template_dict:
    name = item['name']
    template = item['template']
    for index, row in tqdm.tqdm(contextdf.iterrows(), total=len(contextdf)):
        context = row["Context"]
        sentence = row["Sentence"]
        try: 
            embed = np.array(requests.post(
                'https://inf.cl.uni-trier.de/embed/',
                json={'model': MODEL, 'prompt': template + context + '; Sentence: ' + sentence}
                ).json()["response"])
        except Exception as _e:
            logging.warning(_e)
            embed = None
        
        if name=='embed_base':
            embed_base[index] = embed
        if name=='embed_rich':
            embed_rich[index] = embed


100%|██████████| 55/55 [00:11<00:00,  4.76it/s]


In [15]:
contextdf_embeds = contextdf.join(pd.Series(embed_base, name="embed_base")).join(pd.Series(embed_rich, name="embed_rich"))
#contextdf_embeds.to_parquet(f'{CFG.report_dir}/dataset.embeds.parquet')
contextdf_embeds.head()

,Sentence,Context,Set,embed_base,embed_rich
0,I'm going to the bank.,bank,bank-base,"[0.2825756371021271, -0.7305834889411926, -0.6...","[0.40476152300834656, -0.40213334560394287, -0..."
1,I'm going to the bank.,bank-money,bank-base-money,"[0.4423738121986389, -0.6933848857879639, -0.6...","[0.4015294909477234, -0.24794012308120728, -0...."
2,I'm going to the bank.,bank-river,bank-base-river,"[0.31343695521354675, -0.6396633982658386, -0....","[0.3487618863582611, -0.11420242488384247, -0...."
3,I need to visit the bank today.,bank-money,bank-money,"[0.4185699224472046, -0.8129321336746216, -0.7...","[0.3717159330844879, -0.28542259335517883, -0...."
4,I'm heading to the bank to withdraw some cash.,bank-money,bank-money,"[0.35524073243141174, -0.5443449020385742, -0....","[0.322308748960495, -0.14055457711219788, -0.3..."


In [16]:
contextdf_embeds["embeds_n3"] = list(PCA(n_components=3).fit_transform(contextdf_embeds.embed_base.tolist()))
contextdf_embeds["embeds_n3"].head()

0    [-2.5329835575294073, -2.6001338361761697, -0....
1    [-2.239100437384895, -2.45571505517811, -0.561...
2    [-1.9326712449585475, -2.525698063049731, -0.1...
3    [-2.322983161889085, -2.0441169394467007, -0.4...
4    [-2.3292016391646424, -2.8127975483141054, -0....
Name: embeds_n3, dtype: object

In [18]:
plotly.offline.iplot(
    go.Figure(
        data=[
                go.Scatter3d(
                **{
                    dim: group["embeds_n3"].str[idx].tolist() 
                    for idx, dim in enumerate(["x", "y", "z"])
                },
                text = group["Sentence"],
                name=label,
                mode='markers',
                marker={
                    'size': 6,
                    'opacity': 0.8,
                }
            )
            for label, group in contextdf_embeds[~contextdf_embeds['Set'].str.contains('base')].groupby("Set")
        ], 
        layout=go.Layout(
            margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
            height=820
        )
    )
)
